# Bucket Sort for both Sequential and Parallel Implementation

This notebook shows an implementation of Bucket Sort both sequentially and in parallel

In [1]:
!pip install memory_profiler
%load_ext memory_profiler

In [2]:
!pip install line_profiler
%load_ext line_profiler

In [3]:
!pip install psutil

In [4]:
import random

In [5]:
import time
import math
import psutil

In [6]:
from memory_profiler import memory_usage

In [7]:
from threading import Thread

### generate sample array

In [60]:
## array size
n_size = 500000
arr = [None]*n_size
for i in range(0, n_size):
    arr[i] = round(random.uniform(0.0,0.9999),4)

### initialize bucket

In [61]:
## bucket size
n_bucket = int (n_size/100) #Best bucket size is array size however this is assuming
#n_bucket = n_size #Best bucket size is array size however this is assuming 

## Insertion Sort
Insertion sort is the inner sort used since it best performs with small numbers of input

In [62]:
def insertionSort(arr, bucket_no): 
    for i in range(1, len(arr)): 
  
        key = arr[i] 
        j = i-1
        while j >=0 and key < arr[j] : 
                arr[j+1] = arr[j] 
                j -= 1
        arr[j+1] = key 
    return arr

## Sequential Bucket Sort
This method is the one used for sorting all buckets

In [63]:
def seq_bucketSort(arr, size):
    ## record starting time
    start_time = time.clock()
    ## initialize buckets
    buckets = [[] for i in range(size)]
    
    ## Put array in buckets
    for i in range(len(arr)):
        num = size*arr[i]
        buckets[int(num)].append(arr[i])
        
        
    output = []
    
    ## sort individual buckets
    for i in range(len(buckets)):
            insertionSort(buckets[i],i)
        
    ## concatenate the sorted buckets
    for i in range(len(buckets)):
        while len(buckets[i]) > 0:
            output.append(buckets[i].pop(0))
    ## record end time
    end_time = time.clock()
    ## return the time it took to sort the list
    return end_time - start_time

## Main for Sequential Bucket Sort

### sort the array with sequential bucket sort

In [64]:
arr_temp = arr
output = seq_bucketSort(arr_temp, n_bucket)
output

2.4197130000000016

## Parallel Bucket Sort
This method is the one used for sorting all buckets

In [65]:
def par_bucketSort(arr, size):
    ## record the start time
    start_time = time.clock()
    ## initialize buckets
    buckets = [[] for i in range(size)]
    
    ## Put array in buckets
    for i in range(len(arr)):
        num = size*arr[i]
        buckets[int(num)].append(arr[i])
        
        
    output = []
    
    ## sort individual buckets
    for i in range(len(buckets)):
        if(len(buckets[i]) < 1):
            Thread(target=insertionSort, args=(buckets[i],i,)).start()
        
    ## concatenate the sorted buckets
    for i in range(len(buckets)):
        while len(buckets[i]) > 0:
            output.append(buckets[i].pop(0))
    ## record the end time
    end_time = time.clock()
    ## return the time it took to sort the list
    return end_time - start_time

In [66]:
arr_temp = arr
output = par_bucketSort(arr_temp, n_bucket)
output

0.566244999999995

## Distributed Sort
this method of sorting uses a distributed system via request-reply using RESTful API

In [69]:
import requests

def call_sort(arr):
    r = requests.post('http://insertionsortapi.azurewebsites.net/insertionSort2', data = arr)
    arr = r
    return arr

def dis_bucketSort(arr, size):
    ## record the start time
    start_time = time.clock()
    ## initialize buckets
    buckets = [[] for i in range(size)]
    
    ## Put array in buckets
    for i in range(len(arr)):
        num = size*arr[i]
        buckets[int(num)].append(arr[i])
        
        
    output = []
    
    ## sort individual buckets
    for i in range(len(buckets)):
        if(len(buckets[i]) < 1):
            Thread(target=call_sort, args=(buckets[i],)).start()
        
    ## concatenate the sorted buckets
    for i in range(len(buckets)):
        while len(buckets[i]) > 0:
            output.append(buckets[i].pop(0))
    ## record the end time
    end_time = time.clock()
    ## return the time it took to sort the list
    return end_time - start_time

In [70]:
arr_temp = arr
output = dis_bucketSort(arr_temp, n_bucket)
output

0.5268259999999998

## Experiment Automation
The cell below is an automation of the expermient, wherein both implementations are ran multiple times with the same input

In [72]:
## the various size of inputs
n_sizes = [100,500,1000,5000,10000,50000,100000]
for i in range(0,len(n_sizes)):
    n_size = n_sizes[i]
    arr = [None]*n_size
    for i in range(0, n_size):
        arr[i] = round(random.uniform(0.0,0.9999),4)
    print("N SIZE IS ",n_size," _______________________")
    n_bucket = int (n_size/100)
    
    l_seq = []
    l_seq_mem = []
    l_seq_cpu = []
    avg_seq = 0
    avg_seq_mem = 0
    
    for j in range(0,len(n_sizes)):
        arr_temp = arr
        mem_usage = memory_usage((seq_bucketSort, (arr_temp, n_bucket), ))
        l_seq_mem.append(max(mem_usage))
    avg_seq_mem = sum(l_seq_mem)/len(l_seq_mem)
    print('Avg Maximum memory usage: ', avg_seq_mem)
        
    print(" ------------------------------")
    l_par = []
    l_par_mem = []
    l_par_cpu = []
    avg_par = 0
    avg_par_mem = 0
    
    for k in range(0,len(n_sizes)):
        arr_temp = arr
        mem_usage = memory_usage((dis_bucketSort, (arr_temp, n_bucket), ))
        l_par_mem.append(max(mem_usage))
    avg_par_mem = sum(l_par_mem)/len(l_par_mem)
    print('Avg Maximum memory usage: ', avg_par_mem)

N SIZE IS  100  _______________________
Avg Maximum memory usage:  125.9140625
 ------------------------------
Avg Maximum memory usage:  125.9140625
N SIZE IS  500  _______________________
Avg Maximum memory usage:  125.9140625
 ------------------------------
Avg Maximum memory usage:  125.9140625
N SIZE IS  1000  _______________________
Avg Maximum memory usage:  125.9140625
 ------------------------------
Avg Maximum memory usage:  125.9140625
N SIZE IS  5000  _______________________
Avg Maximum memory usage:  125.9140625
 ------------------------------
Avg Maximum memory usage:  125.9140625
N SIZE IS  10000  _______________________
Avg Maximum memory usage:  125.9140625
 ------------------------------
Avg Maximum memory usage:  125.9140625
N SIZE IS  50000  _______________________
Avg Maximum memory usage:  125.9140625
 ------------------------------
Avg Maximum memory usage:  125.9140625
N SIZE IS  100000  _______________________
Avg Maximum memory usage:  125.9140625
 -----------

In [ ]:
r = requests.post('http://insertionsortapi.azurewebsites.net/insertionSort2', data = elements)